# Chat Client 사용

이 노트북은 Azure AI Agent Framework를 사용하여 채팅 클라이언트를 구축하는 방법을 단계별로 안내합니다.

## 학습 목표
- Azure AI Agent Framework의 핵심 개념 이해
- 다양한 Azure 클라이언트 (AzureAIAgentClient, AzureOpenAIChatClient, AzureOpenAIAssistantsClient) 사용법
- Function Calling을 통한 외부 함수 연동
- 스트리밍 응답 처리
- 응답 취소 메커니즘

## 사전 요구사항
1. **uv 설치** (`curl -LsSf https://astral.sh/uv/install.sh | sh`)
2. **Python 3.12 이상**
3. **Azure CLI 설치 및 로그인** (`az login`)
4. **Azure OpenAI 또는 Azure AI 서비스 리소스**
5. **uv를 통한 Python 패키지 관리** (pyproject.toml 기반)

## 환경 설정
이 프로젝트는 uv를 사용하여 Python 환경과 의존성을 관리합니다:
- `pyproject.toml`: 프로젝트 의존성 정의 (Python 3.12+ 요구)
- `uv.lock`: 정확한 버전 고정
- `uv sync --dev`: 개발 환경 의존성 설치

## 1. 환경 설정 및 패키지 설치

uv를 사용하여 필요한 패키지들을 설치합니다. 

**첫 실행 시 터미널에서 다음 명령어를 실행하세요:**
```bash
# Python 3.12로 가상환경 생성
uv venv --python 3.12

# 프로젝트 의존성 설치
uv sync --dev

# Jupyter 커널에 가상환경 등록 (선택사항)
uv run python -m ipykernel install --user --name maf-workshop
```

In [13]:
# uv를 사용한 패키지 설치
import subprocess
import sys
import os

def install_with_uv():
    """uv를 사용하여 패키지를 설치합니다."""
    try:
        # 현재 작업 디렉토리 확인
        current_dir = os.getcwd()
        print(f"현재 작업 디렉토리: {current_dir}")
        
        # pyproject.toml 파일이 있는지 확인
        pyproject_path = os.path.join(current_dir, "pyproject.toml")
        if os.path.exists(pyproject_path):
            print("✅ pyproject.toml 파일을 찾았습니다.")
            
            # uv sync로 의존성 설치
            print("📦 uv sync를 실행하여 의존성을 설치합니다...")
            result = subprocess.run(
                ["uv", "sync", "--dev"],
                capture_output=True,
                text=True,
                cwd=current_dir
            )
            
            if result.returncode == 0:
                print("✅ uv sync 완료!")
                print("설치된 패키지 목록 (일부):")
                
                # 주요 패키지 확인
                key_packages = [
                    "agent-framework",
                    "agent-framework-azure-ai", 
                    "python-dotenv",
                    "azure-identity",
                    "pydantic"
                ]
                
                list_result = subprocess.run(
                    ["uv", "pip", "list"],
                    capture_output=True,
                    text=True,
                    cwd=current_dir
                )
                
                if list_result.returncode == 0:
                    installed_packages = list_result.stdout
                    for pkg in key_packages:
                        if pkg in installed_packages:
                            print(f"  ✅ {pkg}")
                        else:
                            print(f"  ❌ {pkg}")
                
            else:
                print(f"❌ uv sync 실패: {result.stderr}")
                # pip으로 대체 설치 시도
                print("🔄 pip으로 대체 설치를 시도합니다...")
                fallback_install()
                
        else:
            print("❌ pyproject.toml 파일을 찾을 수 없습니다.")
            print("🔄 pip으로 개별 패키지를 설치합니다...")
            fallback_install()
            
    except FileNotFoundError:
        print("❌ uv가 설치되지 않았습니다.")
        print("💡 uv 설치 방법: curl -LsSf https://astral.sh/uv/install.sh | sh")
        print("🔄 pip으로 개별 패키지를 설치합니다...")
        fallback_install()
    except Exception as e:
        print(f"❌ 예기치 않은 오류: {e}")
        fallback_install()

def fallback_install():
    """pip을 사용한 대체 설치"""
    packages = [
        "agent-framework",
        "agent-framework-azure-ai",
        "python-dotenv",
        "azure-identity",
        "pydantic"
    ]
    
    for package in packages:
        try:
            print(f"📦 {package} 설치 중...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} 설치 완료")
        except subprocess.CalledProcessError:
            print(f"❌ {package} 설치 실패")

# 패키지 설치 실행
install_with_uv()

현재 작업 디렉토리: /Users/andy/works/ai/maf-workshop
✅ pyproject.toml 파일을 찾았습니다.
📦 uv sync를 실행하여 의존성을 설치합니다...
✅ uv sync 완료!
설치된 패키지 목록 (일부):
  ✅ agent-framework
  ✅ agent-framework-azure-ai
  ✅ python-dotenv
  ✅ azure-identity
  ✅ pydantic


## 2. 필요한 라이브러리 임포트

Azure AI Agent Framework와 관련 라이브러리들을 임포트합니다.

In [14]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
import os
from random import randint
from typing import Annotated

# Agent Framework 라이브러리
from agent_framework import ChatResponse
from agent_framework.azure import (
    AzureAIAgentClient,
    AzureOpenAIChatClient,
    AzureOpenAIAssistantsClient,
    AzureOpenAIResponsesClient
)

# Azure 인증 (AzureAIAgentClient용)
from azure.identity.aio import AzureCliCredential as AsyncAzureCliCredential

# 유틸리티
from pydantic import BaseModel, Field

print("✅ 모든 라이브러리 임포트 완료!")


✅ 모든 라이브러리 임포트 완료!


## 3. 환경 변수 로드

`.env` 파일에서 Azure 설정 정보를 로드합니다.

`.env` 파일 예시:
```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key
AZURE_OPENAI_API_VERSION=2024-02-01
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4
```

**참고:** 이 노트북은 API 키 방식의 인증을 사용합니다. `AZURE_OPENAI_API_KEY` 환경 변수가 필수입니다.

In [15]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

# 환경 변수 확인
print("환경 변수 확인:")
env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

for var in env_vars:
    value = os.getenv(var)
    if value:
        # API 키는 부분적으로만 표시
        if "API_KEY" in var:
            print(f"✅ {var}: {value[:8]}...")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: 설정되지 않음")

# API 키 확인
api_key = os.getenv("AZURE_OPENAI_API_KEY")
if not api_key:
    print("\n⚠️ 경고: AZURE_OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
    print("클라이언트 생성 시 오류가 발생할 수 있습니다.")
else:
    print("\n✅ API 키가 설정되었습니다. 클라이언트를 사용할 준비가 완료되었습니다.")

환경 변수 확인:
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...
❌ AZURE_OPENAI_API_VERSION: 설정되지 않음
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1

✅ API 키가 설정되었습니다. 클라이언트를 사용할 준비가 완료되었습니다.


## 4. 함수 도구 정의

AI 모델이 호출할 수 있는 사용자 정의 함수를 정의합니다. Function Calling을 통해 AI가 외부 데이터나 서비스에 접근할 수 있습니다.

In [16]:
def get_weather(
    location: Annotated[str, Field(description="날씨를 조회할 도시 이름")],
) -> str:
    """지정된 위치의 날씨 정보를 반환합니다."""
    conditions = ["맑음", "흐림", "비", "눈", "폭풍"]
    temperature = randint(10, 30)
    condition = conditions[randint(0, len(conditions)-1)]
    return f"{location}의 날씨는 {condition}이며, 최고 기온은 {temperature}°C입니다."

def get_time_in_city(
    city: Annotated[str, Field(description="시간을 조회할 도시 이름")],
) -> str:
    """특정 도시의 현재 시간을 반환합니다 (시뮬레이션)."""
    from datetime import datetime, timedelta
    import random
    
    # 임의의 시차를 적용한 시간 생성 (실제 API 대신)
    offset_hours = random.randint(-12, 12)
    city_time = datetime.now() + timedelta(hours=offset_hours)
    return f"{city}의 현재 시간은 {city_time.strftime('%Y-%m-%d %H:%M:%S')}입니다."

# 구조화된 출력을 위한 모델 정의
class WeatherInfo(BaseModel):
    """날씨 정보의 구조화된 출력 모델"""
    location: str = Field(description="도시 이름")
    condition: str = Field(description="날씨 상태")
    temperature: int = Field(description="온도 (섭씨)")
    description: str = Field(description="날씨 설명")

print("✅ 함수 도구 정의 완료!")
print("📝 정의된 함수:")
print("  - get_weather(): 날씨 조회")
print("  - get_time_in_city(): 도시별 시간 조회")
print("  - WeatherInfo: 구조화된 날씨 정보 모델")

✅ 함수 도구 정의 완료!
📝 정의된 함수:
  - get_weather(): 날씨 조회
  - get_time_in_city(): 도시별 시간 조회
  - WeatherInfo: 구조화된 날씨 정보 모델


## 5. Azure AI Agent Client 사용하기

`AzureAIAgentClient`는 Azure AI 서비스와 상호작용하는 가장 직접적인 방법입니다.

In [17]:
async def demo_azure_ai_agent_client():
    """Azure AI Agent Client 기본 사용법을 시연합니다."""
    
    print("🚀 Azure AI Agent Client 데모 시작...")
    
    try:
        # Azure CLI 자격 증명을 사용하여 클라이언트 생성
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            # 1. 간단한 채팅
            print("\n--- 1. 기본 채팅 ---")
            message = "안녕하세요! Azure AI Agent Framework에 대해 간단히 설명해주세요."
            print(f"사용자: {message}")
            
            response = await client.get_response(message)
            print(f"AI: {response}")
            
            # 2. Function Calling 사용
            print("\n--- 2. Function Calling ---")
            message = "암스테르담과 파리의 날씨를 알려주세요."
            print(f"사용자: {message}")
            
            response = await client.get_response(message, tools=get_weather)
            print(f"AI: {response}")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 AZURE_OPENAI_API_KEY 환경 변수를 확인해주세요.")

# 비동기 함수 실행
await demo_azure_ai_agent_client()

🚀 Azure AI Agent Client 데모 시작...

--- 1. 기본 채팅 ---
사용자: 안녕하세요! Azure AI Agent Framework에 대해 간단히 설명해주세요.

--- 1. 기본 채팅 ---
사용자: 안녕하세요! Azure AI Agent Framework에 대해 간단히 설명해주세요.
AI: 안녕하세요! Azure AI Agent Framework에 대해 간단히 설명드리겠습니다.

**Azure AI Agent Framework**는 마이크로소프트가 제공하는 Azure의 새로운 인공지능 에이전트 개발 프레임워크입니다. 이 프레임워크를 이용하면 다양한 AI 에이전트(예: 챗봇, 자동화 도우미 등)를 클라우드 환경에서 보다 쉽고 효율적으로 개발, 배포, 관리할 수 있습니다.

주요 특징은 다음과 같습니다:

1. **멀티 에이전트 지원**  
   여러 개의 에이전트(봇, 플러그인 등)를 한 프로젝트에서 관리할 수 있습니다.

2. **워크플로우 기반**  
   자연어 입력을 통한 워크플로우 구성 및 스킬(능력)을 쉽게 추가할 수 있습니다.

3. **확장성**  
   Microsoft Azure의 다양한 서비스(예: AI 모델, 데이터베이스, 로직 앱 등)와 연동이 쉽고, API 및 플러그인을 통해 기능 확장이 가능합니다.

4. **보안 및 거버넌스 내장**  
   엔터프라이즈 보안과 데이터 거버넌스 기능을 기본적으로 포함하여 안전하고 신뢰성 있는 AI 에이전트 운영이 가능합니다.

5. **Code-first & No-code 개발**  
   개발자가 코드 기반(프로그램)으로 만들 수도 있고, 초보 사용자는 시각적 인터페이스를 활용해 손쉽게 에이전트를 만들 수도 있습니다.

### 활용 예시
- 고객 상담 자동화 챗봇
- 사내 업무 프로세스 자동화 도우미
- 복잡한 데이터 질의 응답 에이전트

### 참고 링크
- [Azure 공식 문서: AI 에이전트 프레임워크](https://learn.microsoft.com/ko-kr

## 6. Azure OpenAI Chat Client 사용하기

`AzureOpenAIChatClient`는 Azure OpenAI 서비스를 직접 사용하는 클라이언트입니다.

In [18]:
async def demo_azure_openai_chat_client():
    """Azure OpenAI Chat Client 사용법을 시연합니다."""
    
    print("🚀 Azure OpenAI Chat Client 데모 시작...")
    
    try:
        # 환경 변수에서 API 키를 읽어 클라이언트 생성
        client = AzureOpenAIChatClient()
        
        # 1. 기본 채팅
        print("\n--- 1. 기본 채팅 ---")
        message = "Python으로 간단한 웹 서버를 만드는 방법을 알려주세요."
        print(f"사용자: {message}")
        
        response = await client.get_response(message)
        print(f"AI: {response}")
        
        # 2. 다중 함수 도구 사용
        print("\n--- 2. 다중 함수 도구 사용 ---")
        message = "서울의 날씨와 현재 시간을 모두 알려주세요."
        print(f"사용자: {message}")
        
        # 여러 함수를 리스트로 전달
        tools = [get_weather, get_time_in_city]
        response = await client.get_response(message, tools=tools)
        print(f"AI: {response}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 Azure OpenAI 리소스 설정을 확인해주세요.")

# 비동기 함수 실행
await demo_azure_openai_chat_client()

🚀 Azure OpenAI Chat Client 데모 시작...

--- 1. 기본 채팅 ---
사용자: Python으로 간단한 웹 서버를 만드는 방법을 알려주세요.
AI: 네, Python으로 간단한 웹 서버를 만드는 방법을 알려드릴게요. 여러 가지 방법이 있지만, 아주 기본적으로 내장 모듈만 쓰는 방법부터 Flask 등 프레임워크 활용까지 예제를 보여드리겠습니다.

---

### 1. **내장 HTTP 서버 (`http.server`) 사용**

아주 간단하게 내장 HTTP 서버를 사용할 수 있습니다.

```python
# Python 3.x 기준
from http.server import SimpleHTTPRequestHandler, HTTPServer

HOST = 'localhost'
PORT = 8000

httpd = HTTPServer((HOST, PORT), SimpleHTTPRequestHandler)
print(f"Serving on http://{HOST}:{PORT}")
httpd.serve_forever()
```

이렇게 실행하면, 현재 폴더의 파일을 웹에서 볼 수 있습니다.  
브라우저에서 <http://localhost:8000> 으로 접속하면 됩니다.

---

### 2. **Flask 사용하기**

좀 더 강력하고 커스텀하게 웹 서버를 만들고 싶다면 [Flask](https://flask.palletsprojects.com/)를 추천합니다.

#### 먼저 설치합니다:
```sh
pip install flask
```

#### 예제 코드:
```python
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello, World!"

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=8000)
```

실행 후,  
브라우저에서 <http://127.0

## 7. 스트리밍 응답 예제

실시간으로 토큰이 생성되는 것을 확인할 수 있는 스트리밍 응답을 구현해봅시다.

In [19]:
async def demo_streaming_response():
    """스트리밍 응답을 시연합니다."""
    
    print("🚀 스트리밍 응답 데모 시작...")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            print("\n--- 스트리밍 응답 (함수 호출 포함) ---")
            message = "도쿄와 런던의 날씨를 비교해서 상세히 설명해주세요."
            print(f"사용자: {message}")
            print("AI: ", end="", flush=True)
            
            # 스트리밍 응답 받기
            async for chunk in client.get_streaming_response(message, tools=get_weather):
                if str(chunk):
                    print(str(chunk), end="", flush=True)
            
            print("\n")  # 줄바꿈
            
            print("\n--- 일반 텍스트 스트리밍 ---")
            message = "인공지능의 미래에 대해 3문단으로 설명해주세요."
            print(f"사용자: {message}")
            print("AI: ", end="", flush=True)
            
            async for chunk in client.get_streaming_response(message):
                if str(chunk):
                    print(str(chunk), end="", flush=True)
            
            print("\n")  # 줄바꿈
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 비동기 함수 실행
await demo_streaming_response()

🚀 스트리밍 응답 데모 시작...

--- 스트리밍 응답 (함수 호출 포함) ---
사용자: 도쿄와 런던의 날씨를 비교해서 상세히 설명해주세요.
AI: 도쿄와도쿄와 런던의 런던의 현재 날 현재 날씨를 비교씨를 비교해드해드리겠습니다리겠습니다.

-.

- 도쿄 도쿄의 날씨는 맑의 날씨는 맑으며,으며, 최고 최고 기 기온은온은 15 15°C입니다°C입니다.. 쾌 쾌적적하고 상하고 상쾌한쾌한 봄 날 봄 날씨로씨로, 야, 야외 활동외 활동이나 산이나 산책에책에 적합 적합한 환경한 환경입니다.입니다. 습도나 습도나 바람 바람 등에는 등에는 큰 이상 큰 이상이 없는이 없는, 좋은, 좋은 날씨 날씨입니다.

입니다.

- 런- 런던의던의 날씨 날씨는 폭는 폭풍이풍이 불고 불고 있으며, 있으며, 최고 기온 최고 기온은 은 30°C30°C입니다.입니다. 평소 평소보다 매우보다 매우 더 더운 날운 날씨에씨에 강 강한 바한 바람과람과 함께 폭 함께 폭풍우풍우가 발생가 발생하고 있어하고 있어 외출 외출 시 주 시 주의의가 필요가 필요합니다.합니다. 비교적 습 비교적 습하고 불하고 불쾌지쾌지수가 높수가 높으며,으며, 기상 기상 악화 악화로로 인해 교 인해 교통이나통이나 일상 일상 활동에 활동에 영향을 미 영향을 미칠 수칠 수 있습니다.

 있습니다.

요약요약하자하자면,면, 도쿄 도쿄는 맑는 맑고 서고 서늘한늘한 반면 반면,, 런던 런던은 매우은 매우 덥 덥고 악고 악천후천후가가 이어지고 이어지고 있습니다. 있습니다. 두 두 도시 모두 도시 모두 날씨에서 날씨에서 큰 차 큰 차이를 보이고이를 보이고 있기 때문에 있기 때문에 외 외출 계획 시 참고하시는 것이 좋겠습니다.출 계획 시 참고하시는 것이 좋겠습니다.


--- 일반 텍스트 스트리밍 ---
사용자: 인공지능의 미래에 대해 3문단으로 설명해주세요.
AI: 


--- 일반 텍스트 스트리밍 ---
사용자: 인공지능의 미래에 대해 3문단으로 설명해주세요.
AI: 인공지능(AI인공지능(AI)의 미래)의 미래는는 삶 삶의 거의의 거의 

## 8. Azure OpenAI Assistants Client

`AzureOpenAIAssistantsClient`는 지속적인 대화와 컨텍스트 관리를 제공합니다.

In [20]:
async def demo_azure_assistants_client():
    """Azure OpenAI Assistants Client 사용법을 시연합니다."""
    
    print("🚀 Azure OpenAI Assistants Client 데모 시작...")
    
    try:
        async with AzureOpenAIAssistantsClient() as client:
            
            print("\n--- Assistant와의 대화 ---")
            
            # 첫 번째 메시지
            message1 = "안녕하세요! 저는 파이썬 개발자입니다. 오늘 뭔가 재미있는 프로젝트를 해보고 싶어요."
            print(f"사용자: {message1}")
            
            response1 = await client.get_response(message1, tools=get_weather)
            print(f"Assistant: {response1}")
            
            # 두 번째 메시지 (컨텍스트 유지)
            message2 = "그럼 현재 날씨에 맞는 프로젝트를 추천해주세요. 서울 날씨를 확인해서 결정해주세요."
            print(f"\n사용자: {message2}")
            
            response2 = await client.get_response(message2, tools=get_weather)
            print(f"Assistant: {response2}")
            
            print("\n💡 Assistant는 이전 대화 내용을 기억하고 있습니다!")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 비동기 함수 실행
await demo_azure_assistants_client()

🚀 Azure OpenAI Assistants Client 데모 시작...

--- Assistant와의 대화 ---
사용자: 안녕하세요! 저는 파이썬 개발자입니다. 오늘 뭔가 재미있는 프로젝트를 해보고 싶어요.
Assistant: 안녕하세요! 파이썬 개발자시군요. 재미있는 프로젝트를 찾고 계시네요. 몇 가지 아이디어를 제안 드릴게요:

1. **날씨 데이터 시각화**: 오픈 API(예: OpenWeatherMap)를 활용해 여러 도시의 실시간 날씨 데이터를 가져와 그래프로 시각화하기.
2. **웹 스크래퍼 만들기**: 관심 있는 웹사이트에서 데이터를 수집해 CSV로 저장하거나 자동 보고서 작성.
3. **챗봇 개발**: 간단한 대화형 챗봇을 만들어 보고, 자연어처리나 텍스트 분석 기능 추가.
4. **추천 시스템**: 영화나 음악, 책 등을 추천하는 개인화 추천 시스템 구현.
5. **이미지/음성 처리**: 간단한 이미지 인식(예: 얼굴 찾기), 혹은 음성 인식 기능을 만들어 보기.
6. **게임 개발**: 파이게임(pygame)으로 작은 미니게임 만들어보기.

어떤 분야가 흥미로우세요? 혹은 위 제안 중에서 관심 가는 것이 있으신가요? 원하는 주제나 목표가 있다면 더 구체적으로 도와드릴 수 있어요!

사용자: 그럼 현재 날씨에 맞는 프로젝트를 추천해주세요. 서울 날씨를 확인해서 결정해주세요.
Assistant: 안녕하세요! 파이썬 개발자시군요. 재미있는 프로젝트를 찾고 계시네요. 몇 가지 아이디어를 제안 드릴게요:

1. **날씨 데이터 시각화**: 오픈 API(예: OpenWeatherMap)를 활용해 여러 도시의 실시간 날씨 데이터를 가져와 그래프로 시각화하기.
2. **웹 스크래퍼 만들기**: 관심 있는 웹사이트에서 데이터를 수집해 CSV로 저장하거나 자동 보고서 작성.
3. **챗봇 개발**: 간단한 대화형 챗봇을 만들어 보고, 자연어처리나 텍스트 분석 기능 추가.
4. **추천 시스템**: 영화나 음악, 책 등을 추천하는 개인화 추천 시스템 구현.

## 9. 구조화된 응답 (Azure Responses Client)

`AzureOpenAIResponsesClient`를 사용하여 JSON 형태의 구조화된 응답을 받아봅시다.

In [21]:
async def demo_azure_responses_client():
    """Azure OpenAI Responses Client 사용법을 시연합니다."""
    
    print("🚀 Azure OpenAI Responses Client 데모 시작...")
    
    try:
        client = AzureOpenAIResponsesClient()
        
        print("\n--- 구조화된 응답 (비스트리밍) ---")
        message = "부산의 날씨를 조회해주세요."
        print(f"사용자: {message}")
        
        # 구조화된 응답 요청
        response = await client.get_response(
            message,
            tools=get_weather,
            response_format=WeatherInfo
        )
        
        print(f"응답 타입: {type(response.value)}")
        print(f"구조화된 응답: {response.value}")
        
        # 개별 필드 접근
        if hasattr(response.value, 'location'):
            print(f"\n📍 위치: {response.value.location}")
            print(f"🌤️  날씨: {response.value.condition}")
            print(f"🌡️  온도: {response.value.temperature}°C")
            print(f"📝 설명: {response.value.description}")
        
        print("\n--- 구조화된 응답 (스트리밍) ---")
        message = "제주도의 날씨 정보를 구조화된 형태로 알려주세요."
        print(f"사용자: {message}")
        
        # 스트리밍으로 구조화된 응답 받기
        response = await ChatResponse.from_chat_response_generator(
            client.get_streaming_response(message, tools=get_weather, response_format=WeatherInfo),
            output_format_type=WeatherInfo,
        )
        
        print(f"스트리밍 구조화된 응답: {response.value}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 비동기 함수 실행
await demo_azure_responses_client()

🚀 Azure OpenAI Responses Client 데모 시작...

--- 구조화된 응답 (비스트리밍) ---
사용자: 부산의 날씨를 조회해주세요.
응답 타입: <class '__main__.WeatherInfo'>
구조화된 응답: location='부산' condition='흐림' temperature=18 description='부산은 현재 흐린 날씨이며, 기온은 18°C입니다.'

📍 위치: 부산
🌤️  날씨: 흐림
🌡️  온도: 18°C
📝 설명: 부산은 현재 흐린 날씨이며, 기온은 18°C입니다.

--- 구조화된 응답 (스트리밍) ---
사용자: 제주도의 날씨 정보를 구조화된 형태로 알려주세요.
응답 타입: <class '__main__.WeatherInfo'>
구조화된 응답: location='부산' condition='흐림' temperature=18 description='부산은 현재 흐린 날씨이며, 기온은 18°C입니다.'

📍 위치: 부산
🌤️  날씨: 흐림
🌡️  온도: 18°C
📝 설명: 부산은 현재 흐린 날씨이며, 기온은 18°C입니다.

--- 구조화된 응답 (스트리밍) ---
사용자: 제주도의 날씨 정보를 구조화된 형태로 알려주세요.
스트리밍 구조화된 응답: location='제주도' condition='눈' temperature=17 description='제주도는 현재 눈이 내리고 있으며, 기온은 17°C입니다.'
스트리밍 구조화된 응답: location='제주도' condition='눈' temperature=17 description='제주도는 현재 눈이 내리고 있으며, 기온은 17°C입니다.'


## 10. 응답 취소 메커니즘

장시간 실행되는 요청을 중간에 취소하는 방법을 알아봅시다.

In [22]:
async def demo_response_cancellation():
    """응답 취소 메커니즘을 시연합니다."""
    
    print("🚀 응답 취소 메커니즘 데모 시작...")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            print("\n--- 시나리오 1: 타임아웃으로 요청 취소 ---")
            
            # 긴 응답을 요청하는 메시지
            message = "인공지능의 역사부터 현재까지의 발전 과정을 매우 상세하게 설명해주세요. 각 시대별로 주요 인물과 기술적 성과를 포함해주세요."
            print(f"사용자: {message}")
            print("⏰ 2초 후 요청을 취소합니다...")
            
            try:
                # 응답 요청을 태스크로 생성
                task = asyncio.create_task(
                    client.get_response(message)
                )
                
                # 2초 대기 후 취소
                await asyncio.sleep(2)
                print("⏹️  요청을 취소합니다...")
                task.cancel()
                
                # 취소된 태스크 대기
                await task
                
            except asyncio.CancelledError:
                print("✅ 요청이 성공적으로 취소되었습니다!")
            
            print("\n--- 시나리오 2: 스트리밍 응답 취소 ---")
            
            message = "세계 각국의 문화와 전통에 대해 매우 길고 상세한 설명을 해주세요."
            print(f"사용자: {message}")
            print("⏰ 스트리밍 중 3초 후 취소합니다...")
            
            try:
                task = asyncio.create_task(
                    stream_with_cancellation(client, message)
                )
                
                await task
                
            except asyncio.CancelledError:
                print("\n✅ 스트리밍 응답이 성공적으로 취소되었습니다!")
    
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

async def stream_with_cancellation(client, message):
    """스트리밍 응답을 받으면서 중간에 취소하는 함수"""
    print("AI: ", end="", flush=True)
    
    start_time = asyncio.get_event_loop().time()
    async for chunk in client.get_streaming_response(message):
        if str(chunk):
            print(str(chunk), end="", flush=True)
        
        # 3초 후 취소
        current_time = asyncio.get_event_loop().time()
        if current_time - start_time > 3:
            print("\n⏹️  스트리밍을 중단합니다...")
            raise asyncio.CancelledError()

# 비동기 함수 실행
await demo_response_cancellation()

🚀 응답 취소 메커니즘 데모 시작...

--- 시나리오 1: 타임아웃으로 요청 취소 ---
사용자: 인공지능의 역사부터 현재까지의 발전 과정을 매우 상세하게 설명해주세요. 각 시대별로 주요 인물과 기술적 성과를 포함해주세요.
⏰ 2초 후 요청을 취소합니다...
⏹️  요청을 취소합니다...
✅ 요청이 성공적으로 취소되었습니다!

--- 시나리오 2: 스트리밍 응답 취소 ---
사용자: 세계 각국의 문화와 전통에 대해 매우 길고 상세한 설명을 해주세요.
⏰ 스트리밍 중 3초 후 취소합니다...
AI: ⏹️  요청을 취소합니다...
✅ 요청이 성공적으로 취소되었습니다!

--- 시나리오 2: 스트리밍 응답 취소 ---
사용자: 세계 각국의 문화와 전통에 대해 매우 길고 상세한 설명을 해주세요.
⏰ 스트리밍 중 3초 후 취소합니다...
AI: 

[2025-12-04 15:59:26 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/context/__init__.py:157 - ERROR] Failed to detach context
Traceback (most recent call last):
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/trace/__init__.py", line 589, in use_span
    yield span
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/observability.py", line 1336, in _get_span
    yield current_span
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/observability.py", line 962, in trace_get_streaming_response
    yield update
GeneratorExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/Users/andy/works/ai/maf-wor


⏹️  스트리밍을 중단합니다...

✅ 스트리밍 응답이 성공적으로 취소되었습니다!


## 11. 에러 처리 및 모범 사례

실제 프로덕션 환경에서 사용할 때의 에러 처리 패턴과 모범 사례를 알아봅시다.

In [23]:
async def demo_error_handling_and_best_practices():
    """에러 처리와 모범 사례를 시연합니다."""
    
    print("🚀 에러 처리 및 모범 사례 데모 시작...")
    
    # 1. 연결 재시도 로직
    print("\n--- 1. 재시도 로직 ---")
    
    async def chat_with_retry(client, message, max_retries=3):
        """재시도 로직이 포함된 채팅 함수"""
        for attempt in range(max_retries):
            try:
                print(f"🔄 시도 {attempt + 1}/{max_retries}")
                response = await client.get_response(message)
                return response
            except Exception as e:
                print(f"❌ 시도 {attempt + 1} 실패: {e}")
                if attempt == max_retries - 1:
                    raise
                await asyncio.sleep(1 * (attempt + 1))  # 지수 백오프
    
    # 2. 컨텍스트 매니저 사용 (리소스 정리)
    print("\n--- 2. 적절한 리소스 관리 ---")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            message = "간단한 인사 메시지를 보내주세요."
            print(f"사용자: {message}")
            
            response = await chat_with_retry(client, message)
            print(f"AI: {response}")
            print("✅ 리소스가 자동으로 정리됩니다.")
            
    except Exception as e:
        print(f"❌ 최종 실패: {e}")
    
    # 3. 함수 도구 에러 처리
    print("\n--- 3. 함수 도구 에러 처리 ---")
    
    def robust_get_weather(
        location: Annotated[str, Field(description="날씨를 조회할 도시 이름")],
    ) -> str:
        """에러 처리가 포함된 날씨 조회 함수"""
        try:
            if not location or location.strip() == "":
                return "❌ 유효한 도시 이름을 입력해주세요."
            
            # 실제 API 호출 시뮬레이션
            conditions = ["맑음", "흐림", "비", "눈"]
            temperature = randint(10, 30)
            condition = conditions[randint(0, len(conditions)-1)]
            
            return f"✅ {location}의 날씨: {condition}, 기온: {temperature}°C"
            
        except Exception as e:
            return f"❌ 날씨 조회 중 오류 발생: {str(e)}"
    
    # 에러 처리가 포함된 함수로 테스트
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            message = "잘못된 도시명과 정상적인 도시명 모두로 날씨를 조회해주세요: '', '서울'"
            print(f"사용자: {message}")
            
            response = await client.get_response(message, tools=robust_get_weather)
            print(f"AI: {response}")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
    
    # 4. 모범 사례 요약
    print("\n--- 📋 모범 사례 요약 ---")
    best_practices = [
        "1. 항상 async with 문을 사용하여 리소스를 자동으로 정리하세요",
        "2. 재시도 로직을 구현하여 일시적인 네트워크 오류에 대응하세요",
        "3. 함수 도구에서는 입력 검증과 에러 처리를 철저히 하세요",
        "4. 환경 변수를 사용하여 민감한 정보를 안전하게 관리하세요",
        "5. 스트리밍 응답 시 적절한 취소 메커니즘을 구현하세요",
        "6. 로깅을 통해 디버깅과 모니터링을 용이하게 하세요"
    ]
    
    for practice in best_practices:
        print(f"✅ {practice}")

# 비동기 함수 실행
await demo_error_handling_and_best_practices()

[2025-12-04 15:59:27 - /opt/homebrew/Cellar/python@3.12/3.12.12/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/base_events.py:1833 - ERROR] Unclosed connection
client_connection: Connection<ConnectionKey(host='aoai-q4u67epqs2xo4.services.ai.azure.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None)>


🚀 에러 처리 및 모범 사례 데모 시작...

--- 1. 재시도 로직 ---

--- 2. 적절한 리소스 관리 ---
사용자: 간단한 인사 메시지를 보내주세요.
🔄 시도 1/3
AI: 안녕하세요! 좋은 하루 되세요. :)
✅ 리소스가 자동으로 정리됩니다.
AI: 안녕하세요! 좋은 하루 되세요. :)
✅ 리소스가 자동으로 정리됩니다.

--- 3. 함수 도구 에러 처리 ---
사용자: 잘못된 도시명과 정상적인 도시명 모두로 날씨를 조회해주세요: '', '서울'

--- 3. 함수 도구 에러 처리 ---
사용자: 잘못된 도시명과 정상적인 도시명 모두로 날씨를 조회해주세요: '', '서울'
AI: 결과는 다음과 같습니다:

- '' (빈 도시명): ❌ 유효한 도시 이름을 입력해주세요.
- '서울': ✅ 서울의 날씨: 맑음, 기온: 18°C

잘못된 도시명은 오류 메시지가 나오고, 정상적인 도시명은 정상적으로 날씨 정보가 표시됩니다.
AI: 결과는 다음과 같습니다:

- '' (빈 도시명): ❌ 유효한 도시 이름을 입력해주세요.
- '서울': ✅ 서울의 날씨: 맑음, 기온: 18°C

잘못된 도시명은 오류 메시지가 나오고, 정상적인 도시명은 정상적으로 날씨 정보가 표시됩니다.

--- 📋 모범 사례 요약 ---
✅ 1. 항상 async with 문을 사용하여 리소스를 자동으로 정리하세요
✅ 2. 재시도 로직을 구현하여 일시적인 네트워크 오류에 대응하세요
✅ 3. 함수 도구에서는 입력 검증과 에러 처리를 철저히 하세요
✅ 4. 환경 변수를 사용하여 민감한 정보를 안전하게 관리하세요
✅ 5. 스트리밍 응답 시 적절한 취소 메커니즘을 구현하세요
✅ 6. 로깅을 통해 디버깅과 모니터링을 용이하게 하세요

--- 📋 모범 사례 요약 ---
✅ 1. 항상 async with 문을 사용하여 리소스를 자동으로 정리하세요
✅ 2. 재시도 로직을 구현하여 일시적인 네트워크 오류에 대응하세요
✅ 3. 함수 도구에서는 입력 검증과 에러 처리를 철저히 하세요
✅ 4. 환경 변수를 사용하여

## 12. 종합 실습

지금까지 학습한 내용을 바탕으로 실제 애플리케이션에서 사용할 수 있는 통합 예제를 만들어봅시다.

In [24]:
class WeatherAssistant:
    """종합적인 날씨 어시스턴트 클래스"""
    
    def __init__(self):
        self.client = None
        self.conversation_history = []
    
    async def __aenter__(self):
        self.client = AzureAIAgentClient(async_credential=AsyncAzureCliCredential())
        await self.client.__aenter__()
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if self.client:
            await self.client.__aexit__(exc_type, exc_val, exc_tb)
    
    def get_weather_extended(
        self,
        location: Annotated[str, Field(description="날씨를 조회할 도시 이름")]
    ) -> str:
        """확장된 날씨 조회 함수"""
        try:
            conditions = ["맑음", "흐림", "비", "눈", "안개"]
            temperature = randint(-5, 35)
            condition = conditions[randint(0, len(conditions)-1)]
            humidity = randint(30, 90)
            wind_speed = randint(0, 20)
            
            weather_info = {
                "location": location,
                "condition": condition,
                "temperature": temperature,
                "humidity": humidity,
                "wind_speed": wind_speed
            }
            
            self.conversation_history.append(f"날씨 조회: {location}")
            
            return (f"{location}의 현재 날씨:\n"
                   f"🌤️ 날씨: {condition}\n"
                   f"🌡️ 기온: {temperature}°C\n"
                   f"💧 습도: {humidity}%\n"
                   f"💨 풍속: {wind_speed}km/h")
        
        except Exception as e:
            return f"❌ {location}의 날씨 조회 실패: {str(e)}"
    
    async def chat(self, message: str, use_streaming: bool = False) -> str:
        """채팅 인터페이스"""
        try:
            self.conversation_history.append(f"사용자: {message}")
            
            if use_streaming:
                print("AI: ", end="", flush=True)
                full_response = ""
                async for chunk in self.client.get_streaming_response(
                    message,
                    tools=self.get_weather_extended
                ):
                    if str(chunk):
                        print(str(chunk), end="", flush=True)
                        full_response += str(chunk)
                print()  # 줄바꿈
                return full_response
            else:
                response = await self.client.get_response(
                    message,
                    tools=self.get_weather_extended
                )
                return str(response)
        
        except Exception as e:
            return f"❌ 오류 발생: {str(e)}"
    
    def get_conversation_summary(self) -> str:
        """대화 요약 반환"""
        return "\n".join(self.conversation_history[-5:])  # 최근 5개 항목

# 종합 실습 실행
async def comprehensive_demo():
    """종합 실습 데모"""
    
    print("🎓 Azure AI Agent Framework 종합 실습")
    print("=" * 50)
    
    async with WeatherAssistant() as assistant:
        
        # 시나리오 1: 기본 날씨 조회
        print("\n📍 시나리오 1: 기본 날씨 조회")
        message1 = "오늘 서울 날씨가 어때요?"
        print(f"사용자: {message1}")
        response1 = await assistant.chat(message1)
        print(f"AI: {response1}")
        
        # 시나리오 2: 다중 도시 비교 (스트리밍)
        print(f"\n🌍 시나리오 2: 다중 도시 날씨 비교 (스트리밍)")
        message2 = "서울, 도쿄, 뉴욕의 날씨를 비교해서 여행하기 좋은 곳을 추천해주세요."
        print(f"사용자: {message2}")
        await assistant.chat(message2, use_streaming=True)
        
        # 시나리오 3: 컨텍스트 기반 질문
        print(f"\n🔗 시나리오 3: 컨텍스트 기반 대화")
        message3 = "그럼 그 중에서 가장 따뜻한 곳은 어디인가요?"
        print(f"사용자: {message3}")
        response3 = await assistant.chat(message3)
        print(f"AI: {response3}")
        
        # 대화 요약 출력
        print(f"\n📋 대화 요약:")
        print(assistant.get_conversation_summary())

# 종합 실습 실행
await comprehensive_demo()

🎓 Azure AI Agent Framework 종합 실습

📍 시나리오 1: 기본 날씨 조회
사용자: 오늘 서울 날씨가 어때요?
AI: 오늘 서울의 날씨는 안개가 끼어 있고, 기온은 -4°C로 매우 추운 편입니다. 습도는 47%이고, 풍속은 2km/h로 약한 바람이 불고 있습니다. 외출하실 때 따뜻하게 입으세요!

🌍 시나리오 2: 다중 도시 날씨 비교 (스트리밍)
사용자: 서울, 도쿄, 뉴욕의 날씨를 비교해서 여행하기 좋은 곳을 추천해주세요.
AI: AI: 오늘 서울의 날씨는 안개가 끼어 있고, 기온은 -4°C로 매우 추운 편입니다. 습도는 47%이고, 풍속은 2km/h로 약한 바람이 불고 있습니다. 외출하실 때 따뜻하게 입으세요!

🌍 시나리오 2: 다중 도시 날씨 비교 (스트리밍)
사용자: 서울, 도쿄, 뉴욕의 날씨를 비교해서 여행하기 좋은 곳을 추천해주세요.
AI: 현재 각현재 각 도시의 도시의 날씨 날씨는는 다음과 다음과 같습니다 같습니다:

:

-- 서울 서울: 비: 비, , 2525°C, 습°C, 습도도  84%, 풍84%, 풍속 속 10km10km/h
/h
- 도- 도쿄:쿄: 맑음 맑음, 19, 19°C,°C, 습도 습도  58%, 풍58%, 풍속 속 0km0km/h
-/h
- 뉴욕 뉴욕:: 눈, 눈, 18°C 18°C, 습도, 습도  6868%, 풍%, 풍속속 11 11km/hkm/h

여

여행행하기하기 좋은 곳 좋은 곳을을 추천 추천하하자자면,
면,
-- 도쿄 도쿄가 맑가 맑고고 온화 온화한 날씨(19°C, 습한 날씨(19°C, 습도도 58%, 58%, 무풍 무풍))로 걷로 걷기나기나 관광 등 관광 등 야 야외 활동외 활동을 하을 하기에 가장기에 가장 적합 적합합니다.
합니다.
-- 서울은 비가 오고 습도가 서울은 비가 오고 습도가 높으며 높으며,
-,
- 뉴욕 뉴욕은 눈은 눈이 오이 오고 있습니다고 있습니다.

.

따따라서라서 오늘 여행을 오늘 여행을 계획하 계획하신다면신다면 “도 “도쿄쿄””를 추천를 추천드립니다!드립니다!

🔗 시

## 13. 마무리 및 추가 학습 자료

🎉 **축하합니다!** Azure AI Agent Framework의 핵심 기능들을 모두 학습하셨습니다.

### 학습한 내용 요약:

1. ✅ **uv 환경 설정** - 현대적인 Python 패키지 관리 (Python 3.12)
2. ✅ **Azure AI Agent Client** - 기본적인 AI 상호작용
3. ✅ **Azure OpenAI Chat Client** - OpenAI 모델 직접 사용
4. ✅ **Function Calling** - 외부 함수와 AI 연동
5. ✅ **스트리밍 응답** - 실시간 토큰 생성
6. ✅ **Assistants Client** - 컨텍스트 유지 대화
7. ✅ **구조화된 응답** - JSON 형태 출력
8. ✅ **응답 취소** - 타임아웃 및 취소 메커니즘
9. ✅ **에러 처리** - 프로덕션 환경 모범 사례
10. ✅ **종합 실습** - 실제 애플리케이션 구현

### uv 환경 관리 명령어 (Python 3.12):

```bash
# Python 3.12로 새 가상환경 생성
uv venv --python 3.12

# 의존성 설치
uv sync --dev

# 새 패키지 추가
uv add package-name

# 개발 의존성 추가  
uv add --dev package-name

# 패키지 업데이트
uv sync --upgrade

# 가상환경 활성화
source .venv/bin/activate  # macOS/Linux
# 또는
.venv\Scripts\activate     # Windows

# Python 버전 확인
uv run python --version  # Python 3.12.x

# Jupyter 노트북 실행
uv run jupyter notebook
```

### 다음 단계:

- 📚 [Agent Framework 공식 문서](https://github.com/microsoft/agent-framework) 
- 🔧 더 복잡한 Function Calling 구현해보기
- 🌐 웹 애플리케이션에 통합하기
- 📊 로깅 및 모니터링 시스템 구축
- 🧪 다양한 AI 모델 실험해보기
- ⚡ [uv 공식 문서](https://docs.astral.sh/uv/) - 현대적인 Python 패키지 관리
- 🐍 Python 3.12의 새로운 기능 활용

### 추가 실습 아이디어:

1. **멀티모달 입력** - 이미지와 텍스트를 함께 처리
2. **대화 메모리** - 장기간 대화 기록 저장
3. **외부 API 연동** - 실제 날씨, 뉴스 API 사용
4. **사용자 인터페이스** - Streamlit, FastAPI 웹앱 구축
5. **uv 스크립트** - pyproject.toml의 [tool.uv.scripts] 활용
6. **Python 3.12 활용** - 새로운 타입 힌트와 성능 개선 기능 사용